# Retrieval and Search

In this tutorial we will put our knowledge database into a search engine which will be used to feed our LLM using a local exectution with Ollama

## Generatiion

In this section we will pass the documents returned by our search engine to our LLM and use it to generate and answer

In [1]:
from openai import OpenAI

In [2]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama'
)

## Using Elastic Search

In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [4]:
es_client.info()

ObjectApiResponse({'name': '2b3f5ca55987', 'cluster_name': 'docker-cluster', 'cluster_uuid': '_DKVGVXzQE-kX-7c3BSkHQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # Allowsto filter
        }
    }
}

index_name = "course-questions"
response = es_client.indices.create(index=index_name, body=index_settings)

response

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/GtMhJAILRiO6QFnzRb1liQ] already exists')

In [6]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [48]:
# We bring the documents to elastic search using tqdm progressbar
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:49<00:00, 19.27it/s]


In [7]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # The hat means three times more importance
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [13]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
query = 'I just disovered the course. Can I still join it?'
rag(query)

' If you\'re just discovering the course, it seems like there is a possibility for late registration or joining after starting date in terms of submitting homework assignments as long as they align within their respective deadlines mentioned in our FAQs:\n\n"Yes, even if you don\'t register, you\'re still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in final projects. So do not leave everything until last minute."\n\nAs per your inquiry about following the course after it has finished: \n\n"Yes, we keep all materials from a concluded class so you can continue at your own pace later on. You might want to consider preparing for subsequent cohorts or starting work on your final capstone project if this is applicable."'

In [16]:
print(_)

 If you're just discovering the course, it seems like there is a possibility for late registration or joining after starting date in terms of submitting homework assignments as long as they align within their respective deadlines mentioned in our FAQs:

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in final projects. So do not leave everything until last minute."

As per your inquiry about following the course after it has finished: 

"Yes, we keep all materials from a concluded class so you can continue at your own pace later on. You might want to consider preparing for subsequent cohorts or starting work on your final capstone project if this is applicable."
